In [312]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2022, day=15)

def parses(input):
    template = 'Sensor at x={:d}, y={:d}: closest beacon is at x={:d}, y={:d}'
    return [parse.parse(template, line).fixed
            for line in input.strip().split('\n')]
        
data = parses(puzzle.input_data)

In [313]:
sample = parses("""Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3""")

In [314]:
sample

[(2, 18, -2, 15),
 (9, 16, 10, 16),
 (13, 2, 15, 3),
 (12, 14, 10, 16),
 (10, 20, 10, 16),
 (14, 17, 10, 16),
 (8, 7, 2, 10),
 (2, 0, 2, 10),
 (0, 11, 2, 10),
 (20, 14, 25, 17),
 (17, 20, 21, 22),
 (16, 7, 15, 3),
 (14, 3, 15, 3),
 (20, 1, 15, 3)]

In [315]:
def merge_intervals(intervals):
    merged_intervals = []
    for s, e in sorted(intervals):
        if not merged_intervals or s > merged_intervals[-1][1] + 1:
            merged_intervals.append([s,e])
        else:
            merged_intervals[-1][1] = max(merged_intervals[-1][1], e)
    return merged_intervals

In [316]:
def intervals_at_y(data, y):
    intervals = []
    for sx, sy, bx, by in data:
        r = abs(sx-bx) + abs(sy-by)
        ry = r - abs(sy-y)
        if ry >= 0:
            intervals.append([sx-ry,sx+ry])
    return merge_intervals(intervals)

In [317]:
def solve_a(data, y=2_000_000):
    intervals = intervals_at_y(data, y)
    beacons_y = len(set(bx for _, _, bx, by in data if by==y))
    return sum((e-s+1) for s, e in intervals) - beacons_y

In [318]:
solve_a(sample, 10)

26

In [110]:
%%timeit
solve_a(data)

KeyboardInterrupt: 

In [133]:
solve_a(data)

4985193

In [134]:
intervals_at_y(data, 0)

[[-264840, 5468872]]

In [ ]:
4_000_000

In [176]:
def solve_b_bruteforce(data, bounds=4_000_000):
    
    for y in range(bounds+1):
        intervals = intervals_at_y(data, y)
        for s, e in intervals:
            if s >= 0 and e <= bounds:
                return (s-1) * 4_000_000 + y
            if e+1 <= bounds:
                return (e+1) * 4_000_000 + y
            if s > bounds:
                break

In [146]:
solve_b(sample, 20)

56000011

In [148]:
%%time
solve_b(data)

CPU times: user 32.8 s, sys: 309 ms, total: 33.1 s
Wall time: 33.4 s


11583882601918

In [180]:
def solve_b_smt(data, bounds=4_000_000):
    # We can just offload the hard work to a solver,
    # specifying the bounds as well as the fact that 
    # the point we are looking for must like outside
    # ALL of the diamonds defined by the sensor-beacon 
    # pairs. Runs in a similar amount of time to bruteforce
    import z3
    s = z3.Solver()
    x = z3.Int('x')
    y = z3.Int('y')
    for v in (x,y):
        s.add(0 <= v)
        s.add(v <= bounds)
    
    def manhattan(p1, p2):
        abs = lambda x: z3.If(x >= 0, x, -x)
        return sum(abs(x1 - x2) for x1, x2 in zip(p1, p2))
    
    for sx, sy, bx, by in data:
        r = abs(sx-bx) + abs(sy-by)
        s.add(manhattan((x,y), (sx,sy)) > r)
    if s.check():
        m = s.model()
        return m[x].as_long() * 4_000_000 + m[y].as_long()

In [207]:
ps = np.arange(14).reshape((7,2))

In [208]:
ps

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11],
       [12, 13]])

In [212]:
(ps[:,0] >= 2) & ((ps[:,0] <= 10))
(ps[:,1] >= 2) & ((ps[:,1] <= 10))

array([False,  True,  True,  True,  True, False, False])

In [242]:
data

[(3523437, 2746095, 3546605, 2721324),
 (282831, 991087, 743030, -87472),
 (1473740, 3283213, 1846785, 3045894),
 (1290563, 46916, 743030, -87472),
 (3999451, 15688, 3283637, -753607),
 (1139483, 2716286, 1846785, 3045894),
 (3137614, 2929987, 3392051, 3245262),
 (2667083, 2286333, 2126582, 2282363),
 (3699264, 2920959, 3546605, 2721324),
 (3280991, 2338486, 3546605, 2721324),
 (833202, 92320, 743030, -87472),
 (3961416, 2485266, 3546605, 2721324),
 (3002132, 3500345, 3392051, 3245262),
 (2482128, 2934657, 1846785, 3045894),
 (111006, 2376713, 354526, 3163958),
 (424237, 2718408, 354526, 3163958),
 (3954504, 3606495, 3392051, 3245262),
 (2275050, 2067292, 2333853, 2000000),
 (1944813, 2557878, 2126582, 2282363),
 (2227536, 2152792, 2126582, 2282363),
 (3633714, 1229193, 3546605, 2721324),
 (1446898, 1674290, 2333853, 2000000),
 (3713985, 2744503, 3546605, 2721324),
 (2281504, 3945638, 1846785, 3045894),
 (822012, 3898848, 354526, 3163958),
 (89817, 3512049, 354526, 3163958),
 (2594265,

In [307]:
@njit
def solve_b_perimeter(data, bounds=4_000_000):
    sensors = []
    for sx, sy, bx, by in data:
        r = abs(sx-bx) + abs(sy-by)
        sensors.append((sx,sy,r))
    
    for sx, sy, r in sensors:
        for x_sign, y_sign in [(1,1),(1,-1),(-1,1),(-1,-1)]:
            R = r+1
            for i in range(R):
                x, y = sx+R*x_sign-i, sy+y_sign*i
                if 0 <= x <= bounds and 0 <= y <= bounds:
                    for x2, y2, r2 in sensors:
                        if abs(x-x2) + abs(y-y2) <= r2:
                            break
                    else:
                        return x * 4_000_000 + y

In [308]:
def solve_b_smart(data, bounds=4_000_000):
    # As there is only one missing value, it's going to be just outside the
    # boundaries of at least two scanners (unless we're incredibly unlucky and it's
    # right on the bounds of the 0-4_000_000 square
    
    # Like the perimeter solution, the idea is to only test points at the diamond 
    # perimeters. Even smarter than the perimeter solution is to only consider 
    # points at the intersection of two line segments.
    
    # To simplify the logic, line segments can be simplified to lines since we 
    # are going to do the scanner checks anyways
    
    positive_offset = [] # y = x + b
    negative_offset = [] # y = -x + b
    scanners = []
    for sx, sy, bx, by in data:
        r = abs(sx-bx) + abs(sy-by)
        scanners.append((sx,sy,r))
        R = r + 1
        # lines that contain top of diamond (sx+R, sy)
        positive_offset.append(sy-sx-R)
        negative_offset.append(sy+sx+R)
        # lines that contain bottom of diamond (sx-R, sy)
        positive_offset.append(sy-sx+R)
        negative_offset.append(sy+sx-R)
    
    for pos in positive_offset:
        for neg in negative_offset:
            x, y = (neg-pos) // 2, (pos+neg) // 2
            if 0 <= x <= bounds and 0 <= y <= bounds:
                if all(abs(sx-x)+abs(sy-y) > r for sx, sy, r in scanners):
                    return x * 4_000_000 + y
                    

In [309]:
solve_b_smart(sample, 20)

56000011

In [311]:
solve_b_smart(data)

11583882601918

    so

In [296]:
solve_b(sample, 20)

/Users/josejavier/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/numba/core/ir_utils.py:2119: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'data' of function 'solve_b'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-295-da84af07a491>", line 2:
@njit
def solve_b(data, bounds=4_000_000):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


56000011

In [299]:
%%time
solve_b(data)

CPU times: user 284 ms, sys: 2.45 ms, total: 287 ms
Wall time: 287 ms


11583882601918

In [302]:
from scipy.spatial.distance import cdist

def solve_b(data, bounds=4_000_000):
    sensors = []
    radii = []
    for sx, sy, bx, by in data:
        r = abs(sx-bx) + abs(sy-by)
        sensors.append((sx,sy))
        radii.append(r)
    sensors = np.array(sensors)
    radii = np.array(radii)
    
    for (sx, sy), r in zip(sensors, radii):
        R = r + 1
        ramp = np.arange(R)
        sides = [
            (sx+R-ramp, sy+ramp),
            (sx+R-ramp, sy-ramp),
            (sx-R+ramp, sy+ramp),
            (sx-R+ramp, sy-ramp),
        ]
        for xs, ys in sides:
            side = np.stack([xs, ys],-1)
            within = (side[:,0] >= 0) & (side[:,0] <= bounds) & (side[:,1] >= 0) & (side[:,1] <= bounds)
            side = side[within]

            ds = cdist(side,sensors,metric='cityblock')
            valid = (ds > radii).all(axis=1)
            side = side[valid]
            if len(side) > 0:
                x, y = side[0]
                return x * 4_000_000 + y

In [303]:
solve_b(sample,20)

56000011

In [304]:
%%time
solve_b(data)

CPU times: user 3.63 s, sys: 1.24 s, total: 4.87 s
Wall time: 4.91 s


11583882601918

In [179]:
solve_b_smt(sample, 20)

[x >= 0,
 x <= 20,
 y >= 0,
 y <= 20,
 0 +
 If(x - 2 >= 0, x - 2, -(x - 2)) +
 If(y - 18 >= 0, y - 18, -(y - 18)) >
 7,
 0 +
 If(x - 9 >= 0, x - 9, -(x - 9)) +
 If(y - 16 >= 0, y - 16, -(y - 16)) >
 1,
 0 +
 If(x - 13 >= 0, x - 13, -(x - 13)) +
 If(y - 2 >= 0, y - 2, -(y - 2)) >
 3,
 0 +
 If(x - 12 >= 0, x - 12, -(x - 12)) +
 If(y - 14 >= 0, y - 14, -(y - 14)) >
 4,
 0 +
 If(x - 10 >= 0, x - 10, -(x - 10)) +
 If(y - 20 >= 0, y - 20, -(y - 20)) >
 4,
 0 +
 If(x - 14 >= 0, x - 14, -(x - 14)) +
 If(y - 17 >= 0, y - 17, -(y - 17)) >
 5,
 0 +
 If(x - 8 >= 0, x - 8, -(x - 8)) +
 If(y - 7 >= 0, y - 7, -(y - 7)) >
 9,
 0 +
 If(x - 2 >= 0, x - 2, -(x - 2)) +
 If(y - 0 >= 0, y - 0, -(y - 0)) >
 10,
 0 +
 If(x - 0 >= 0, x - 0, -(x - 0)) +
 If(y - 11 >= 0, y - 11, -(y - 11)) >
 3,
 0 +
 If(x - 20 >= 0, x - 20, -(x - 20)) +
 If(y - 14 >= 0, y - 14, -(y - 14)) >
 8,
 0 +
 If(x - 17 >= 0, x - 17, -(x - 17)) +
 If(y - 20 >= 0, y - 20, -(y - 20)) >
 6,
 0 +
 If(x - 16 >= 0, x - 16, -(x - 16)) +
 If(y -

56000011

In [177]:
%%time
solve_b(data)

CPU times: user 32.6 s, sys: 296 ms, total: 32.9 s
Wall time: 33.2 s


11583882601918

In [151]:



s.add(x <= bounds)